In [1]:
import torch
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration
from datasets import load_dataset
import soundfile as sf

import sys
sys.path.append("/home/shoinoue/Git/cuhksz-phd/sho_util/pyfiles/")
from sound import play_audio

/mntcephfs/lab_data/shoinoue/miniconda3/envs/cuhk/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
model_path = "/mntcephfs/data/audiow/shoinoue/Model/hf_hub/models--facebook--s2t-large-librispeech-asr/snapshots/a4b4750ad1425acda0dbd1daa9188d5fd7872491"
model = Speech2TextForConditionalGeneration.from_pretrained(model_path).to("cuda")
processor = Speech2TextProcessor.from_pretrained(model_path)

In [20]:
class FacebookASR():
    def __init__(self, fs=16000, model_path="/mntcephfs/data/audiow/shoinoue/Model/hf_hub/models--facebook--s2t-large-librispeech-asr/snapshots/a4b4750ad1425acda0dbd1daa9188d5fd7872491"):
        self.model = Speech2TextForConditionalGeneration.from_pretrained(model_path).to("cuda")
        self.processor = Speech2TextProcessor.from_pretrained(model_path)
        self.fs = fs
        
    def get_sample(self, file):
        x, _ = librosa.load(file)
        write_wav("temp.wav", self.fs, x)
        speech, _ = sf.read("temp.wav")
        return speech
    
    def get_transcription(self, file):
        speech = self.get_sample(file)
        input_features = self.processor(
            speech,
            sampling_rate=self.fs,
            return_tensors="pt"
        ).input_features.cuda()  # Batch size 1
        generated_ids = self.model.generate(inputs=input_features)
        transcription = self.processor.batch_decode(generated_ids)
        return transcription

In [5]:
file = '/mntcephfs/lab_data/shoinoue/Dataset/L2-ARCTIC/ASI/wav/arctic_b0539.wav'

In [11]:
from scipy.io.wavfile import write as write_wav
import librosa

def get_sample(file, fs=16000):
    x, _ = librosa.load(file)
    write_wav("temp.wav", fs, x)
    speech, _ = sf.read("temp.wav")
    return speech

speech = get_sample(file)
input_features = processor(
    speech,
    sampling_rate=16_000,
    return_tensors="pt"
).input_features.cuda()  # Batch size 1
generated_ids = model.generate(inputs=input_features)
transcription = processor.batch_decode(generated_ids)

/mntcephfs/lab_data/shoinoue/miniconda3/envs/cuhk/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [12]:
transcription

['you were making them talk sharp ruth charged him']